
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>


# 2.1 DEMO: Implementing Delta Sharing (Databricks-to-Databricks)

## Overview
In this demo, we will implement Delta Sharing to securely share data between Databricks workspaces. This demonstration is split into two parts:

**Provider Notebook (This Notebook):** We will create a Unity Catalog with sample customer and sales data, configure Delta Sharing by creating a share, set up a recipient organization, and generate an activation link for secure access.

**Recipient Notebook:** The receiving organization (using Databricks) will mount the share in a local catalog and query the shared tables directly without copying data, and perform analytics using the shared datasets.

This demo showcases the Databricks-to-Databricks (D2D) sharing pattern, where both the provider and recipient use Databricks workspaces. You'll see how Delta Sharing enables secure, live data sharing without data duplication or complex ETL processes.

### Learning Objectives
By the end of this demo, you will be able to:
1. Create and configure Unity Catalog resources (catalogs, schemas, and tables) suitable for Delta Sharing.
2. Implement Delta Sharing by creating a share and adding tables to it for external access.
3. Create and configure recipients to control who can access your shared data.
4. Understand the provider's role in the Delta Sharing architecture and how to manage shared data access.
5. Verify and monitor shared data configurations using SQL commands in the provider workspace.

## Background

**Scenario:**
You are a data provider company that wants to share customer analytics data with partner organizations using Delta Sharing. In this notebook, you'll set up the provider side of Delta Sharing by creating the data, configuring shares, and generating access credentials for your recipients.

In [0]:
%run ../Includes/Classroom-Setup-2

## Step 1: Create Catalog and Schema

We'll create a Unity Catalog and schema to hold our data that will be shared.

In [0]:
-- Create the catalog if it doesn't exist
CREATE CATALOG IF NOT EXISTS ${c.catalog_name}
COMMENT 'Provider catalog for Delta Sharing demo';

-- Show catalogs to verify creation
SHOW CATALOGS LIKE '${c.catalog_name}';

In [0]:
-- Use the catalog and create schema
USE CATALOG ${c.catalog_name};

CREATE SCHEMA IF NOT EXISTS ${c.schema_name}
COMMENT 'Schema containing sales and customer data for sharing';

-- Show schemas to verify
SHOW SCHEMAS IN ${c.catalog_name};

## Step 2: Create Sample Tables

Let's create some realistic sample data that represents customer and sales information.

In [0]:
-- Create customers table
CREATE OR REPLACE TABLE ${c.catalog_name}.${c.schema_name}.customers (
  customer_id INT,
  customer_name STRING,
  email STRING,
  country STRING,
  signup_date DATE,
  customer_segment STRING
)
COMMENT 'Customer master data';

-- Insert sample data
INSERT INTO ${c.catalog_name}.${c.schema_name}.customers VALUES
  (1, 'Acme Corp', 'contact@acme.com', 'USA', '2023-01-15', 'Enterprise'),
  (2, 'TechStart Inc', 'info@techstart.com', 'Canada', '2023-02-20', 'SMB'),
  (3, 'Global Solutions', 'hello@globalsol.com', 'UK', '2023-03-10', 'Enterprise'),
  (4, 'Innovate Ltd', 'contact@innovate.com', 'Australia', '2023-04-05', 'Mid-Market'),
  (5, 'Data Dynamics', 'team@datadyn.com', 'Germany', '2023-05-12', 'Enterprise'),
  (6, 'Cloud First', 'info@cloudfirst.com', 'USA', '2023-06-18', 'SMB'),
  (7, 'NextGen Analytics', 'contact@nextgen.com', 'Singapore', '2023-07-22', 'Mid-Market'),
  (8, 'Digital Transform', 'hello@digitrans.com', 'France', '2023-08-30', 'Enterprise');

SELECT * FROM ${c.catalog_name}.${c.schema_name}.customers;

In [0]:
-- Create sales transactions table
CREATE OR REPLACE TABLE ${c.catalog_name}.${c.schema_name}.sales_transactions (
  transaction_id INT,
  customer_id INT,
  product_name STRING,
  quantity INT,
  unit_price DECIMAL(10,2),
  total_amount DECIMAL(10,2),
  transaction_date DATE,
  region STRING
)
COMMENT 'Sales transaction data';

-- Insert sample sales data
INSERT INTO ${c.catalog_name}.${c.schema_name}.sales_transactions VALUES
  (101, 1, 'Data Platform License', 5, 1000.00, 5000.00, '2024-01-15', 'North America'),
  (102, 2, 'Analytics Suite', 2, 500.00, 1000.00, '2024-01-20', 'North America'),
  (103, 3, 'Data Platform License', 10, 1000.00, 10000.00, '2024-02-05', 'EMEA'),
  (104, 1, 'Storage Add-on', 3, 200.00, 600.00, '2024-02-10', 'North America'),
  (105, 4, 'Analytics Suite', 5, 500.00, 2500.00, '2024-02-15', 'APAC'),
  (106, 5, 'Data Platform License', 15, 1000.00, 15000.00, '2024-03-01', 'EMEA'),
  (107, 3, 'Storage Add-on', 8, 200.00, 1600.00, '2024-03-10', 'EMEA'),
  (108, 6, 'Analytics Suite', 1, 500.00, 500.00, '2024-03-15', 'North America'),
  (109, 7, 'Data Platform License', 7, 1000.00, 7000.00, '2024-04-01', 'APAC'),
  (110, 8, 'Storage Add-on', 10, 200.00, 2000.00, '2024-04-05', 'EMEA');

SELECT * FROM ${c.catalog_name}.${c.schema_name}.sales_transactions;

## Step 3: Create a Delta Share

Now we'll create a share and add our tables to it. A share is a logical grouping of tables that can be shared with recipients.

In [0]:
-- Create a new share
CREATE SHARE IF NOT EXISTS ${c.share_name}
COMMENT 'Share containing customer and sales data for partner analysis';

-- View all shares
SHOW SHARES;

In [0]:
-- Add tables to the share
ALTER SHARE ${c.share_name} 
ADD TABLE ${c.catalog_name}.${c.schema_name}.customers
COMMENT 'Customer master data';

ALTER SHARE ${c.share_name} 
ADD TABLE ${c.catalog_name}.${c.schema_name}.sales_transactions
COMMENT 'Sales transaction history';

-- View what's in the share
SHOW ALL IN SHARE ${c.share_name};

## Step 4: Create a Recipient (Databricks-to-Databricks)

Create a recipient for Databricks-to-Databricks sharing. The recipient will receive an activation link to access the shared data.

In [0]:
-- Create a Databricks-to-Databricks recipient
CREATE RECIPIENT IF NOT EXISTS ${c.d2d_recipient}
COMMENT 'Recipient for D2D Delta Sharing demo';

-- View all recipients
SHOW RECIPIENTS;

In [0]:
-- Grant the share to the recipient
GRANT SELECT ON SHARE ${c.share_name} TO RECIPIENT ${c.d2d_recipient};

-- Describe the recipient to get the activation link
DESCRIBE RECIPIENT ${c.d2d_recipient};

## Step 5: Get Activation Link

**Important:** Copy the `activation_link` from the output above and share it with the recipient. The recipient will use this link to activate the share in their workspace.

The activation link will look like:
```
https://accounts.cloud.databricks.com/...
```

## Step 6: Monitor and Manage the Share

Let's review what we've created and explore monitoring capabilities.

In [0]:
-- View share details
DESCRIBE SHARE ${c.share_name};

-- List all tables in the share
SHOW ALL IN SHARE ${c.share_name};

-- View recipient details
DESCRIBE RECIPIENT ${c.d2d_recipient};

## Summary

✅ **What we accomplished:**

1. Created a Unity Catalog and schema for our data
2. Created sample customers and sales_transactions tables
3. Created a Delta Share and added tables to it
4. Created a Databricks-to-Databricks recipient
5. Granted the share to the recipient
6. Generated an activation link for the recipient

**Next Steps:**
- Share the activation link with the recipient workspace
- The recipient will use the activation link to create a catalog in their workspace
- The recipient can then query the shared data

**Key Concepts:**
- **Share**: A logical grouping of tables that can be shared
- **Recipient**: An entity that receives access to a share
- **Activation Link**: A one-time link used to activate the share in the recipient workspace
- **D2D Sharing**: Direct sharing between two Databricks workspaces using Unity Catalog

&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>